In [1]:
import os.path

from utils import get_completion_from_messages
from text_extraction import TextExtraction
from synthetic_data_generation import create_opinionated
import json

import pandas as pd

from tqdm import tqdm

# TODOS
1. select some random json files from the all_data_articles
2. extract the context fro each of the citation in the text
3. change the context to opinionated with open ai

In [3]:
filename = "A_Colonial_Celebrity_in_the_New_Attention_Economy_Cecil_Rhodess_Cape-to-Cairo_Telegraph_and_Railway_Negotiations_in_1899.json"

filepath = "../all_data_articles/"
synthetic_filepath = "../synthetic_data/"

with open(os.path.join(filepath, filename), "r", encoding="utf-8") as file:
    article_dict = json.load(file)

previous_context_tokens = 70
following_context_tokens = 30
    
footnotes = article_dict["footnotes"]

text_extractor = TextExtraction(
    article_dict,
    previous_context_tokens=previous_context_tokens,
    following_context_tokens=following_context_tokens,
    previous_context_sentences=None,
    following_context_sentences=None,
    previous_whole_paragraph=False,
    following_whole_paragraph=False,
    till_previous_citation=None,
    till_following_citation=None,
    footnote_text=False,
    footnote_mask=True,
)

generated_contexts = []

for footnote_number in tqdm(footnotes):
    context = text_extractor.generate_context(int(footnote_number))
    
    context_and_footnote = context + "\n\n" + "CITATION:\n" + footnotes[footnote_number]
    
    generated_context = create_opinionated(context_and_footnote)
    generated_contexts.append(generated_context)

generated_contexts_df = pd.DataFrame(columns=["footnote_number", "generated_context", "footnote_text", "label"])

generated_contexts_df["footnote_number"] = footnotes.keys()
generated_contexts_df["generated_context"] = generated_contexts
generated_contexts_df["footnote_text"] = footnotes.values()
generated_contexts_df["label"] = [1] * len(footnotes)

generated_contexts_df.to_csv(os.path.join(synthetic_filepath, filename.replace(".json", ".csv")), index=False)

100%|██████████| 135/135 [14:02<00:00,  6.24s/it]
